# Assignment 3 - more pitching#
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Hoenigman ####
#### Due: Wednesday, March 6 by 3pm ####

**Use any of the datasets we've learned about so far to complete this assignment.** 

**Submit your notebook to the Assignment 3 link on Canvas. Include all csv files that you generate with your submission.**

## Pitching ###
In Assignment 2, you evaluated 5 Rockies pitchers by comparing pitch selection and movement of their pitches at home and away. In this next assignment, you will also evaluate pitchers in different environments or at different times. The specifics of how you do that are up to you. Your assignment needs to fulfill the following high-level objectives.

### 1. Pitchers to compare ###
(10 pts) You need pitchers that you will compare. Examples include groups of pitchers compared to other groups, an individual pitcher compared to another individual, or one pitcher compared to himself at different times in his career. 

### 2. Reason for your comparison ###
(10 pts) You need to have a reason for why you think the comparison is meaningful. For example, you might compare the star pitcher from a hitters park to the star pitcher at a pitchers park. This comparison could be meaningful to evaluate the effect of the run-scoring environment on their approach to pitching. A comparison that wouldn't be meaningful is average pitcher A compared to average pitcher B, both on average teams. That's not a result that anyone would care about. 

### 3. Basis for comparison ###
(30 pts) You need to describe how the pitchers will be compared and then implement your comparison. Examples of comparison include pitch movement and selection, BABIP, BABIP on specific pitches, OBP in different environments. Choose at least three criteria for pitcher comparison. 

### 4. Data for comparison ###
(10 pts) Focus on starting pitchers only because you will have more data. Use 3-5 years of data for each pitcher or time period. A pitcher will need to have at least 3 years of data to be used in this assignment.

### 5. Analysis ###
(20 pts) Your comparison needs to include a basic statistical analysis, such as t-tests, z-tests, and/or confidence intervals. You also need at least one visual presentation of your data, such as a histogram, bar chart, or line graph. Your visual should be appropriate for the type of data being displayed.

### 6. Presentation ###
(20 pts) You need a 2-page writeup explaining what you did and your results. Your presentation should have clearly marked sections for 

* Introduction
* Problem Definition
* Data
* Results

### Other notes ###
* Proofread your writeup for typos and proper grammar. 
* If you compare pitch movement, use the statcast fields for vx0, vz0, ax, az, plate_x, plate_z variables instead of pfx_x, pfx_z, I'm not sure what those do.
* Include all code and the writeup together in one notebook.

Data gathering and filtering starts here. All portions of this assignment will be addressed in detail in the writeup, including visualizations.

In [78]:
#from pybaseball import statcast

#pitches = statcast('2016-01-01', '2018-12-20')
#pitches_2016 = statcast('2016-04-02', '2016-11-02')

#pitches_2016_2018 = pd.concat([pitches, pitches_2016], ignore_index=True).drop("index", axis=1)
#pitches_2016_2018.to_csv("pitches_2016_2018.csv")

In [130]:
from pybaseball import pitching
import pandas as pd

p = pitching()

In [54]:
p_2016 = p.loc[(p["yearID"] == 2016) & (p["lgID"] == "NL")]

# looking for pitchers that had a near-average ERA in the NL in 2016
avg_era = p_2016["ERA"].mean()
avg_pitchers = p_2016.loc[(p_2016["ERA"] <= avg_era + 0.5) &
                          (p_2016["ERA"] >= avg_era - 0.5)].copy()

In [58]:
avg_lookup = playerid_reverse_lookup(avg_pitchers["playerID"], key_type="bbref")

Gathering player lookup table. This may take a moment.


In [65]:
avg_pitchers = pd.merge(avg_pitchers, avg_lookup[['key_bbref', 'key_mlbam']],
         left_on="playerID", right_on='key_bbref', how='left')

In [76]:
# the pitches of our average pitchers
apitchers_pitches = pitches_2016_2018.loc[(pitches_2016_2018["pitcher"].isin(avg_pitchers["key_mlbam"]))].copy()

In [77]:
# write the data out to csv
#apitchers_pitches.to_csv("apitchers_pitches.csv")

In [82]:
apitchers_pitches["game_year"] = apitchers_pitches["game_date"].apply(lambda x: x.year)

In [135]:
# pitched in 2016, 2017, and 2018 more than a rough average
avg_pitches_per_year = apitchers_pitches.groupby(["game_year", "pitcher"]).size().mean()
avg_pitches_per_year

pitched_a_lot = apitchers_pitches.groupby(["pitcher"]).filter(lambda x: x["game_year"].nunique() == 3 and
                                             x.size >= 2*avg_pitches_per_year)["pitcher"].unique()

In [137]:
pal_pitches = apitchers_pitches.loc[(apitchers_pitches["pitcher"].isin(pitched_a_lot))]
pal_percentages = pal_pitches.groupby(["pitcher", "pitch_type"]).size() / pal_pitches.groupby(["pitcher"]).size()
pal_percentages[pal_percentages > 0.35]

pitcher   pitch_type
424144.0  SL            0.443478
434538.0  FT            0.416461
501822.0  FF            0.462299
543339.0  FF            0.490687
543901.0  SI            0.627246
571578.0  SL            0.355619
592662.0  FF            0.508916
596133.0  FF            0.581915
605151.0  FF            0.699310
606131.0  FT            0.536979
606983.0  SL            0.411184
608379.0  FF            0.477693
608566.0  FF            0.486622
612672.0  FF            0.493851
621107.0  FF            0.408193
623184.0  FF            0.480897
625643.0  FF            0.607188
640463.0  FF            0.354857
dtype: float64

In [ ]:
ff_pitchers = [501822.0, 543339.0]

In [111]:
avg_pitchers.loc[(avg_pitchers["key_mlbam"].isin(pitched_a_lot))]

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,HBP,BK,BFP,GF,R,SH,SF,GIDP,key_bbref,key_mlbam
2,bradlar01,2016,1,ARI,NL,8,9,26,26,0,...,4.0,2,638.0,0.0,84,2.0,7.0,7.0,bradlar01,605151.0
7,chacijh01,2016,1,ATL,NL,1,2,5,5,0,...,0.0,0,117.0,0.0,17,2.0,1.0,0.0,chacijh01,468504.0
8,chenwe02,2016,1,MIA,NL,5,5,22,22,0,...,3.0,0,520.0,0.0,69,3.0,4.0,10.0,chenwe02,612672.0
9,coleaj01,2016,1,WAS,NL,1,2,8,8,0,...,2.0,0,168.0,0.0,24,0.0,3.0,0.0,coleaj01,595918.0
10,corbipa01,2016,1,ARI,NL,5,13,36,24,0,...,5.0,0,701.0,6.0,109,6.0,5.0,15.0,corbipa01,571578.0
14,eflinza01,2016,1,PHI,NL,3,5,11,11,2,...,1.0,0,272.0,0.0,42,1.0,4.0,5.0,eflinza01,621107.0
18,gloveko01,2016,1,WAS,NL,2,0,19,0,0,...,1.0,0,83.0,4.0,12,0.0,0.0,1.0,gloveko01,606983.0
21,hatchch02,2016,1,LAN,NL,5,4,37,0,0,...,1.0,0,181.0,10.0,26,1.0,0.0,3.0,hatchch02,501822.0
23,hudsoda01,2016,1,ARI,NL,3,2,70,0,0,...,4.0,0,268.0,17.0,40,0.0,0.0,3.0,hudsoda01,543339.0
28,liriafr01,2016,1,PIT,NL,6,11,21,21,0,...,7.0,0,523.0,0.0,76,7.0,5.0,13.0,liriafr01,434538.0
